In [1]:
import pickle as pkl

In [2]:
file = open('data/data_for_checking_loaders.pkl', 'rb')
data = pkl.load(file)
file.close()

In [3]:
data

,formula,framework,composition,n_atoms_unrelaxed,n_atoms_opt,unrelaxed_cryst_id,unrelaxed_struct,opt_cryst_id,opt_struct,es_job_id,dft_energy,dft_energy_per_atom,dft_e_hull
0,Mn1O3Sb1,MnSbO3,"{'sites': {'A': ['Mn'], 'B': ['Sb'], 'X': ['O'...",5,40,137458332,[[7.27035675e-01 2.69595108e-19 0.00000000e+00...,151855989,"[[1.03668026 0. 0. ] Mn, [1.036...",51310512,-256.179186,-6.40448,0.413235
1,Cd1Ir1O3,IrCdO3,"{'sites': {'A': ['Ir'], 'B': ['Cd'], 'X': ['O'...",5,40,137458333,[[1.24166199e+00 1.75102687e-13 1.37403279e-14...,151918212,"[[3.21059535 1.93133496 1.93133496] Cd, [3.210...",51789972,-192.720815,-4.81802,0.993241
2,In1O3V1,VInO3,"{'sites': {'A': ['V'], 'B': ['In'], 'X': ['O']...",5,40,137458334,"[[1.10564595 0. 0. ] V, [2.2124...",151944346,"[[3.24304772 1.81490016 1.81490016] In, [3.243...",51895902,-249.674827,-6.241871,0.607629
3,Cu1O3Pb1,PbCuO3,"{'sites': {'A': ['Pb'], 'B': ['Cu'], 'X': ['O'...",5,40,137458335,[[ 2.15266114e-01 -2.17156472e-15 2.73644150e...,151882716,"[[2.25103269 1.88292668 1.88292668] Cu, [2.251...",51634093,-197.893852,-4.947346,0.340135
4,Hf1O3Ta1,TaHfO3,"{'sites': {'A': ['Ta'], 'B': ['Hf'], 'X': ['O'...",5,40,137458336,[[9.77294571e-01 2.24271159e-18 0.00000000e+00...,151860518,"[[1.94019034 2.11648954 2.11648954] Hf, [1.940...",51342231,-378.934765,-9.473369,0.792048
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3027,Be1Ce1O3,CeBeO3,"{'sites': {'A': ['Ce'], 'B': ['Be'], 'X': ['O'...",5,40,137465002,"[[2. 2. 2.] Be, [0. 0. 0.] Ce, [2. 2. 0.] O, [...",151928319,"[[1.80008086 1.80008086 1.80008086] Be, [1.800...",51853610,-306.891835,-7.672296,0.417001
4617,Ba7Co3La1Ni5O24,Ba0.875La0.125Co0.375Ni0.625O3,"{'sites': {'A': ['Ba', 'La'], 'B': ['Co', 'Ni'...",40,40,201883163,"[[8.000000e-06 8.000000e-06 4.000008e+00] Ba, ...",262017661,"[[0.02443428 0.02443428 4.06009044] Ba, [-0.01...",76676368,-225.50894,-5.637724,0.1881
7659,Ga1Na2Nb7O24Pb6,Na0.250Pb0.750Ga0.125Nb0.875O3,"{'sites': {'A': ['Na', 'Pb'], 'B': ['Ga', 'Nb'...",40,40,205585591,"[[8.e-06 8.e-06 8.e-06] Na, [2.000008 2.000008...",209624698,"[[1.97903265 2.03626107 2.03626107] Ga, [-0.21...",66018039,-296.983843,-7.424596,0.123989
1356,Ca1O3Pd1,CaPdO3,"{'sites': {'A': ['Ca'], 'B': ['Pd'], 'X': ['O'...",5,40,137459768,[[ 4.03700469e-01 -3.30211422e-18 -1.84657167e...,151883150,[[4.83670348e-01 4.58413030e-06 4.58413030e-06...,51654435,-227.154364,-5.678859,0.140539


In [4]:
from processing.utils import filter_data_by_properties,select_structures

In [5]:
data = filter_data_by_properties(data,"dft_e_hull")

In [6]:
data_unrelaxed = select_structures(data,"unrelaxed")

In [7]:
data_relaxed = select_structures(data,"relaxed")

In [8]:
from processing.interpolation.Interpolation import *

In [9]:
data_unrelaxed = apply_interpolation(data_unrelaxed,"dft_e_hull")

In [10]:
data_unrelaxed

,formula,framework,composition,n_atoms_unrelaxed,n_atoms_opt,unrelaxed_cryst_id,unrelaxed_struct,opt_cryst_id,opt_struct,es_job_id,dft_energy,dft_energy_per_atom,dft_e_hull,structure,dft_e_hull_interp,dft_e_hull_diff
6787,Ba1O24Rb7Ti7V1,Ba0.125Rb0.875Ti0.875V0.125O3,"{'sites': {'A': ['Ba', 'Rb'], 'B': ['V', 'Ti']...",40,40,205329467,"[[0. 0. 0.] Ba, [2. 2. 2.] V, [2. 2. 0.] O, [0...",213265840,"[[-0.04035941 -0.04035941 -0.04035941] Ba, [ 1...",69733340,-277.678013,-6.94195,0.234979,"[[-0.04035941 -0.04035941 -0.04035941] Ba, [ 1...",0.189777,0.045201
9726,La8Li4Mo4O24,LaLi0.500Mo0.500O3,"{'sites': {'A': ['La'], 'B': ['Li', 'Mo'], 'X'...",40,40,261839885,"[[3.90014939 7.84244649 6.30405396] La, [7.900...",261895880,"[[4.10394133 7.79910124 6.8948906 ] La, [8.021...",76546822,-300.865176,-7.521629,0.061616,"[[4.10394133 7.79910124 6.8948906 ] La, [8.021...",0.286327,-0.224711
7132,K4O24Pd6Rb4Ti2,K0.500Rb0.500Pd0.750Ti0.250O3,"{'sites': {'A': ['K', 'Rb'], 'B': ['Pd', 'Ti']...",40,40,205581929,"[[8.000000e-06 8.000000e-06 4.000008e+00] Rb, ...",252322577,[[1.25736912e-03 1.25736912e-03 1.02043710e-05...,72945225,-207.672065,-5.191802,0.204386,[[1.25736912e-03 1.25736912e-03 1.02043710e-05...,0.144915,0.059471
8412,Ba8O24Sc4Ti4,BaSc0.500Ti0.500O3,"{'sites': {'A': ['Ba'], 'B': ['Sc', 'Ti'], 'X'...",40,40,205591139,"[[6.000008 2.000008 6.000008] Ba, [8.000000e-0...",209638031,"[[6.18453646 2.06151903 6.20163514] Ba, [6.184...",66019586,-308.160546,-7.704014,0.061506,"[[6.18453646 2.06151903 6.20163514] Ba, [6.184...",0.039036,0.022470
9712,Ba8Co4Mo4O24,BaCo0.500Mo0.500O3,"{'sites': {'A': ['Ba'], 'B': ['Co', 'Mo'], 'X'...",40,40,261839866,"[[6.000008 2.000008 6.000008] Ba, [6.000008 6....",262135472,"[[6.12076058 2.04026033 6.12076058] Ba, [6.120...",76676393,-266.930822,-6.673271,0.04587,"[[6.12076058 2.04026033 6.12076058] Ba, [6.120...",0.127175,-0.081304
6467,Ga2Na6O24Rb2Ru6,Na0.750Rb0.250Ga0.250Ru0.750O3,"{'sites': {'A': ['Rb', 'Na'], 'B': ['Ga', 'Ru'...",40,40,205328993,"[[0. 0. 0.] Rb, [2. 2. 2.] Ga, [2. 2. 0.] O, [...",209635943,"[[1.9306789 1.9306789 1.93017062] Ga, [1.930...",66019502,-236.523847,-5.913096,0.140415,"[[1.9306789 1.9306789 1.93017062] Ga, [1.930...",0.173263,-0.032848
6422,Ba3La5O24Rh4Sc4,Ba0.375La0.625Rh0.500Sc0.500O3,"{'sites': {'A': ['Ba', 'La'], 'B': ['Rh', 'Sc'...",40,40,205328922,"[[8.e-06 8.e-06 8.e-06] Ba, [2.000008 2.000008...",251413595,"[[ 0.08084181 -0.03145358 -0.03145358] Ba, [0....",72249684,-307.838545,-7.695964,0.047878,"[[ 0.08084181 -0.03145358 -0.03145358] Ba, [0....",0.146158,-0.098280
5619,Ba1La7O24Rh7Sc1,Ba0.125La0.875Rh0.875Sc0.125O3,"{'sites': {'A': ['Ba', 'La'], 'B': ['Sc', 'Rh'...",40,40,205327668,"[[0. 0. 0.] Ba, [2. 2. 2.] Sc, [2. 2. 0.] O, [...",252539831,"[[0.04235672 0.04235672 0.04235672] Ba, [0.235...",72945561,-296.665427,-7.416636,0.155793,"[[0.04235672 0.04235672 0.04235672] Ba, [0.235...",0.198375,-0.042582
6666,Ga2Na1O24Rb7Ru6,Na0.125Rb0.875Ga0.250Ru0.750O3,"{'sites': {'A': ['Na', 'Rb'], 'B': ['Ga', 'Ru'...",40,40,205329293,"[[8.e-06 8.e-06 8.e-06] Na, [2.000008 2.000008...",208170001,"[[1.96467545 1.98130837 1.98130837] Ga, [1.964...",65084855,-232.65037,-5.816259,0.21097,"[[1.96467545 1.98130837 1.98130837] Ga, [1.964...",0.238211,-0.027241
9521,Ca8Fe4Ni4O24,CaFe0.500Ni0.500O3,"{'sites': {'A': ['Ca'], 'B': ['Fe', 'Ni'], 'X'...",40,40,222998288,"[[6.000008 2.000008 2.000008] Ca, [8.000000e-0...",253622152,"[[5.75036498 1.93569576 1.93569576] Ca, [5.750...",73354879,-231.231449,-5.780786,0.149435,"[[5.75036498 1.93569576 1.93569576] Ca, [5.750...",0.149399,0.000036


In [11]:
from processing.dataloader.dataloader import get_dataloaders

In [12]:
train_loader, val_loader, test_loader = get_dataloaders(data_unrelaxed,"dft_e_hull","CGCNN",4)

100%|██████████| 18/18 [00:00<00:00, 1142.05it/s]


In [13]:
for batch_id, data in enumerate(train_loader):
    print(data)

[[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[[5.6052e-45, 1.1603e-36, 3.3985e-29,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [4.2039e-45, 8.3109e-37, 2.5252e-29,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [4.2039e-45, 8.3109e-37, 2.5252e-29,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[5.6052e-45, 1.1603e-36, 3.3985e-29,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [4.2039e-45, 8.3109e-37, 

[[tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[[0.0000e+00, 0.0000e+00, 3.4713e-38,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 3.4711e-38,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 

In [14]:
train_loader, val_loader, test_loader = get_dataloaders(data_unrelaxed,"dft_e_hull","e3nn",4)

100%|██████████| 3/3 [00:00<00:00, 28.13it/s]


In [15]:
for batch_id, data in enumerate(train_loader):
    print(data)

DataBatch(x=[160, 118], edge_index=[2, 6716], pos=[160, 3], lattice=[4, 3, 3], symbol=[4], comp=[4], z=[160, 92], edge_shift=[6716, 3], edge_vec=[6716, 3], edge_len=[4], target=[4, 1], idx=[4, 1], batch=[160], ptr=[5])
DataBatch(x=[160, 118], edge_index=[2, 8160], pos=[160, 3], lattice=[4, 3, 3], symbol=[4], comp=[4], z=[160, 92], edge_shift=[8160, 3], edge_vec=[8160, 3], edge_len=[4], target=[4, 1], idx=[4, 1], batch=[160], ptr=[5])
DataBatch(x=[120, 118], edge_index=[2, 5274], pos=[120, 3], lattice=[3, 3, 3], symbol=[3], comp=[3], z=[120, 92], edge_shift=[5274, 3], edge_vec=[5274, 3], edge_len=[3], target=[3, 1], idx=[3, 1], batch=[120], ptr=[4])


In [16]:
train_loader, val_loader,test_loader = get_dataloaders(data_unrelaxed,"dft_e_hull","e3nn_contrastive",4)

100%|██████████| 18/18 [00:00<00:00, 2796.72it/s]


In [17]:
for batch_id, data in enumerate(train_loader):
    print(data.comp)


['Ca8Fe4Ni4O24', 'Ca8Fe4Ni4O24', 'Cr6K7O24Ti2Y1', 'K4O24Pd6Rb4Ti2', 'La8Li4Mo4O24']
['Ga2Na1O24Rb7Ru6', 'Ba3La5O24Rh4Sc4', 'Na2O24Rb6Ru7Ti1', 'Ba1La7O24Rh7Sc1']
['Ba8Co4Mo4O24', 'Ba8Co4Mo4O24']


In [18]:
train_loader, val_loader,test_loader = get_dataloaders(data_unrelaxed,"dft_e_hull","Painn",4)


100%|██████████| 18/18 [00:00<00:00, 2015.68it/s]

{'crystal_id': [9726, 7132, 9712, 6422, 5619, 6666, 9521, 9520, 4956, 5419, 9713], 'nxyz': [array([[5.70000000e+01, 4.10394133e+00, 7.79910124e+00, 6.89489060e+00],
       [5.70000000e+01, 8.02149541e+00, 3.88154725e+00, 6.89489060e+00],
       [5.70000000e+01, 3.79251782e+00, 4.01491191e+00, 6.84816658e+00],
       [5.70000000e+01, 7.88813067e+00, 8.11052484e+00, 6.94161462e+00],
       [5.70000000e+01, 4.49752400e+00, 4.27283681e+00, 2.11855139e+00],
       [5.70000000e+01, 4.01911147e-01, 1.77223881e-01, 2.02510336e+00],
       [5.70000000e+01, 4.18380738e+00, 4.90940577e-01, 2.07182737e+00],
       [5.70000000e+01, 2.66253308e-01, 4.40849457e+00, 2.07182737e+00],
       [3.00000000e+00, 6.13670712e+00, 6.13786719e+00, 4.51582019e-01],
       [3.00000000e+00, 2.04109427e+00, 2.04225434e+00, 3.58133982e-01],
       [3.00000000e+00, 6.04883776e+00, 2.13012370e+00, 4.04858001e-01],
       [3.00000000e+00, 2.13128376e+00, 6.04767769e+00, 4.04858001e-01],
       [4.20000000e+01, 6.107238

TypeError: Dataset.__init__() got an unexpected keyword argument 'stack'